In [1]:
from time import time
import random

In [2]:
epochs_duration = []
epochs_duration.append(time())
epochs_duration.append(time())

### Evaluation function 

In [3]:
def evaluate_solution(solution, books_scores):
    books_scanned = set()
    points = 0
    number_of_libraries = solution[0]
    for i in range(1, number_of_libraries+1):
        number_of_books = solution[i][1]
        for j in range(number_of_books):
            book_to_add = solution[i][2][j]
            if book_to_add not in books_scanned:
                books_scanned.add(book_to_add)
    solution_score = 0
    for i in books_scanned:
        solution_score += books_scores[i]
        
    return solution_score

### Get complete solution from list of solutions

In [4]:
def get_complete_solution(solution_indices, libraries, days_number, books_scores):
    time_pointer = days_number
    used_books = set()
    complete_solution = [len(solution_indices)]
    for i in solution_indices:
        time_pointer -= libraries[i][0][1]
        books_per_day = libraries[i][0][2]
        available_books = list(libraries[i][1])
        particular_solution = [i, 0, []]
        available_books.sort(key = lambda x: books_scores[x])
        for x in available_books:
            if len(particular_solution[2]) > time_pointer * books_per_day:
                break
            if x not in used_books:
                particular_solution[2].append(x)
                particular_solution[1] += 1
                used_books.add(x)
        particular_solution[2] = tuple(particular_solution[2])
        complete_solution.append(tuple(particular_solution))
    return tuple(complete_solution)

### Initial population

In [5]:
def get_random_solution(libraries_number, days_number, libraries, books_scores):
    libraries_to_use = list(libraries)
    solution_indices = []
    time_pointer = 0
    while True:
        random_index = random.randint(0, len(libraries_to_use) - 1)
        random_library = libraries_to_use[random_index]
        del libraries_to_use[random_index]
        if time_pointer + random_library[0][1] > days_number:
            break                                 
        solution_indices.append(random_index)
        time_pointer += random_library[0][1]
    return get_complete_solution(solution_indices, libraries, days_number, books_scores)
        
    
def get_initial_population(population_size, libraries_number, days_number, libraries, books, timestamp, max_duration):
    population = []
    for i in range(int(population_size / 2)):
        population.append(get_random_solution(libraries_number, days_number, libraries, books))
        population.append(get_random_solution(libraries_number, days_number, libraries, books))
        if time() - timestamp >= max_duration / 2:
            return (i * 2, population)
    return (population_size, population)

### Mutation

In [6]:
def mutate(solution, libraries, days_number, books_scores):
    solution_indices = [solution[i][0] for i in range(1, len(solution))]
    for _ in range(3):
        nucleotide_a = random.randint(0, len(solution_indices) - 1)
        nucleotide_b = random.randint(0, len(solution_indices) - 1)
        solution_indices[nucleotide_a], solution_indices[nucleotide_b] = solution_indices[nucleotide_b], solution_indices[nucleotide_a]
    return get_complete_solution(solution_indices, libraries, days_number, books_scores)

def do_mutations(population, libraries, days_number, books_scores):
    mutated_solutions = []
    for solution in population:
        mutated_solutions.append(mutate(solution, libraries, days_number, books_scores))
    return mutated_solutions

## Crossover

In [7]:
def get_children(parent_a, parent_b, libraries, days_number, books_scores):
    parent_a_indices = [parent_a[i][0] for i in range(1, len(parent_a))]
    parent_b_indices = [parent_b[i][0] for i in range(1, len(parent_b))]
    split_point = random.randint(0, min(len(parent_a_indices), len(parent_b_indices)) - 1)
    parent_a_indices[:split_point], parent_b_indices[split_point:] = parent_b_indices[:split_point], parent_a_indices[split_point:]
    for parent in (parent_a_indices, parent_b_indices):
        time_pointer = 0
        for i in range(len(parent)):
            if time_pointer >= days_number:
                parent_a_indices = parent[:i]
                break
            time_pointer += libraries[parent[i]][1]
        time_pointer = 0
    child_a = get_complete_solution(parent_a_indices, libraries, days_number, books_scores)
    child_b = get_complete_solution(parent_a_indices, libraries, days_number, books_scores)
    return (child_a, child_b)
    

In [8]:
def get_offspring(population, libraries, days_number, books_scores):
    offspring = []
    random.shuffle(population)
    for i in range(len(population), 2):
        offspring.append(get_children(population[i], population[i + 1], libraries, days_number, books_scores))
    return tuple(offspring)

# Main

<font color="blue">__Specify how long program is able to run here (in seconds)__</blue>

In [9]:
max_duration = 300

<font color="blue">__Specify population size here__</font>

In [10]:
population_size = 80

### Read data

<font color=blue>__Specify path to input file here__</font>

In [11]:
file = open("instances/f_libraries_of_the_world.txt", "r")

Load data from source file to the program

In [12]:
books_number, libraries_number, days_number = [int(x) for x in file.readline().split()]
books_scores = tuple([int(x) for x in file.readline().split()])
libraries = [None] * libraries_number
for i in range(libraries_number):
    libraries[i] = (tuple([int(x) for x in file.readline().split()]), tuple([int(x) for x in file.readline().split()]))
libraries = tuple(libraries) 

Get initial population

In [13]:
population_size, population = get_initial_population(population_size, libraries_number, days_number, libraries, books_scores, epochs_duration[0], max_duration)

In [14]:
#for x in population:
#    print(evaluate_solution(x, books_scores))

main loop - epochs <br>


In [15]:
epoch = 0
while epochs_duration[0] + max_duration >= time() + (epochs_duration[-1] - epochs_duration[-2]) * 2:
    mutations = do_mutations(population, libraries, days_number, books_scores)
    offspring = get_offspring(population, days_number, libraries, books_scores)
    population.extend(offspring)
    population.extend(mutations)
    population.sort(key = lambda x: evaluate_solution(x, books_scores), reverse = True)
    population = population[:population_size]
    epochs_duration.append(time())
    print("epoch: " + str(epoch) + "; timestamp: " + str(round(time() - epochs_duration[0], 2)) +  "; epoch's duration: ", str(round(epochs_duration[-1] - epochs_duration[-2], 2)) + "; best solution: " + str(evaluate_solution(population[0], books_scores)))
    epoch += 1

epoch: 0; timestamp: 0.84; epoch's duration:  0.84; best solution: 1376772
epoch: 1; timestamp: 1.09; epoch's duration:  0.25; best solution: 1376772
epoch: 2; timestamp: 1.38; epoch's duration:  0.29; best solution: 1376772
epoch: 3; timestamp: 1.7; epoch's duration:  0.33; best solution: 1376772
epoch: 4; timestamp: 2.05; epoch's duration:  0.35; best solution: 1376772
epoch: 5; timestamp: 2.4; epoch's duration:  0.34; best solution: 1376772
epoch: 6; timestamp: 2.74; epoch's duration:  0.34; best solution: 1376772
epoch: 7; timestamp: 3.09; epoch's duration:  0.35; best solution: 1376772
epoch: 8; timestamp: 3.46; epoch's duration:  0.37; best solution: 1376772
epoch: 9; timestamp: 3.85; epoch's duration:  0.39; best solution: 1376772
epoch: 10; timestamp: 4.21; epoch's duration:  0.37; best solution: 1376772
epoch: 11; timestamp: 4.63; epoch's duration:  0.41; best solution: 1376772
epoch: 12; timestamp: 5.03; epoch's duration:  0.41; best solution: 1376772
epoch: 13; timestamp: 5.

epoch: 107; timestamp: 39.52; epoch's duration:  0.35; best solution: 1376772
epoch: 108; timestamp: 39.87; epoch's duration:  0.35; best solution: 1376772
epoch: 109; timestamp: 40.25; epoch's duration:  0.38; best solution: 1376772
epoch: 110; timestamp: 40.66; epoch's duration:  0.41; best solution: 1376772
epoch: 111; timestamp: 41.04; epoch's duration:  0.38; best solution: 1376772
epoch: 112; timestamp: 41.42; epoch's duration:  0.38; best solution: 1376772
epoch: 113; timestamp: 41.77; epoch's duration:  0.35; best solution: 1376772
epoch: 114; timestamp: 42.12; epoch's duration:  0.35; best solution: 1376772
epoch: 115; timestamp: 42.48; epoch's duration:  0.36; best solution: 1376772
epoch: 116; timestamp: 42.82; epoch's duration:  0.34; best solution: 1376772
epoch: 117; timestamp: 43.18; epoch's duration:  0.35; best solution: 1376772
epoch: 118; timestamp: 43.52; epoch's duration:  0.35; best solution: 1376772
epoch: 119; timestamp: 43.87; epoch's duration:  0.35; best solu

epoch: 213; timestamp: 81.71; epoch's duration:  0.37; best solution: 1376772
epoch: 214; timestamp: 82.09; epoch's duration:  0.38; best solution: 1376772
epoch: 215; timestamp: 82.46; epoch's duration:  0.37; best solution: 1376772
epoch: 216; timestamp: 82.83; epoch's duration:  0.37; best solution: 1376772
epoch: 217; timestamp: 83.2; epoch's duration:  0.37; best solution: 1376772
epoch: 218; timestamp: 83.56; epoch's duration:  0.35; best solution: 1376772
epoch: 219; timestamp: 83.91; epoch's duration:  0.35; best solution: 1376772
epoch: 220; timestamp: 84.28; epoch's duration:  0.37; best solution: 1376772
epoch: 221; timestamp: 84.64; epoch's duration:  0.36; best solution: 1376772
epoch: 222; timestamp: 85.0; epoch's duration:  0.36; best solution: 1376772
epoch: 223; timestamp: 85.36; epoch's duration:  0.35; best solution: 1376772
epoch: 224; timestamp: 85.71; epoch's duration:  0.36; best solution: 1376772
epoch: 225; timestamp: 86.06; epoch's duration:  0.35; best soluti

epoch: 318; timestamp: 119.29; epoch's duration:  0.36; best solution: 1376772
epoch: 319; timestamp: 119.65; epoch's duration:  0.35; best solution: 1376772
epoch: 320; timestamp: 120.0; epoch's duration:  0.35; best solution: 1376772
epoch: 321; timestamp: 120.36; epoch's duration:  0.36; best solution: 1376772
epoch: 322; timestamp: 120.71; epoch's duration:  0.35; best solution: 1376772
epoch: 323; timestamp: 121.11; epoch's duration:  0.4; best solution: 1376772
epoch: 324; timestamp: 121.54; epoch's duration:  0.43; best solution: 1376772
epoch: 325; timestamp: 121.94; epoch's duration:  0.4; best solution: 1376772
epoch: 326; timestamp: 122.31; epoch's duration:  0.37; best solution: 1376772
epoch: 327; timestamp: 122.68; epoch's duration:  0.36; best solution: 1376772
epoch: 328; timestamp: 123.04; epoch's duration:  0.36; best solution: 1376772
epoch: 329; timestamp: 123.42; epoch's duration:  0.38; best solution: 1376772
epoch: 330; timestamp: 123.77; epoch's duration:  0.35;

epoch: 423; timestamp: 160.28; epoch's duration:  0.35; best solution: 1376772
epoch: 424; timestamp: 160.63; epoch's duration:  0.35; best solution: 1376772
epoch: 425; timestamp: 160.99; epoch's duration:  0.36; best solution: 1376772
epoch: 426; timestamp: 161.33; epoch's duration:  0.34; best solution: 1376772
epoch: 427; timestamp: 161.71; epoch's duration:  0.37; best solution: 1376772
epoch: 428; timestamp: 162.43; epoch's duration:  0.72; best solution: 1376772
epoch: 429; timestamp: 162.9; epoch's duration:  0.47; best solution: 1376772
epoch: 430; timestamp: 163.4; epoch's duration:  0.5; best solution: 1376772
epoch: 431; timestamp: 163.9; epoch's duration:  0.5; best solution: 1376772
epoch: 432; timestamp: 164.3; epoch's duration:  0.4; best solution: 1376772
epoch: 433; timestamp: 164.67; epoch's duration:  0.36; best solution: 1376772
epoch: 434; timestamp: 165.01; epoch's duration:  0.34; best solution: 1376772
epoch: 435; timestamp: 165.4; epoch's duration:  0.39; best

epoch: 528; timestamp: 200.47; epoch's duration:  0.36; best solution: 1376772
epoch: 529; timestamp: 200.82; epoch's duration:  0.35; best solution: 1376772
epoch: 530; timestamp: 201.18; epoch's duration:  0.37; best solution: 1376772
epoch: 531; timestamp: 201.59; epoch's duration:  0.4; best solution: 1376772
epoch: 532; timestamp: 202.01; epoch's duration:  0.42; best solution: 1376772
epoch: 533; timestamp: 202.4; epoch's duration:  0.39; best solution: 1376772
epoch: 534; timestamp: 202.77; epoch's duration:  0.37; best solution: 1376772
epoch: 535; timestamp: 203.3; epoch's duration:  0.53; best solution: 1376772
epoch: 536; timestamp: 203.8; epoch's duration:  0.51; best solution: 1376772
epoch: 537; timestamp: 204.22; epoch's duration:  0.41; best solution: 1376772
epoch: 538; timestamp: 204.73; epoch's duration:  0.51; best solution: 1376772
epoch: 539; timestamp: 205.17; epoch's duration:  0.44; best solution: 1376772
epoch: 540; timestamp: 205.66; epoch's duration:  0.49; 

epoch: 632; timestamp: 239.3; epoch's duration:  0.35; best solution: 1376772
epoch: 633; timestamp: 239.69; epoch's duration:  0.39; best solution: 1376772
epoch: 634; timestamp: 240.05; epoch's duration:  0.36; best solution: 1376772
epoch: 635; timestamp: 240.4; epoch's duration:  0.35; best solution: 1376772
epoch: 636; timestamp: 240.75; epoch's duration:  0.35; best solution: 1376772
epoch: 637; timestamp: 241.1; epoch's duration:  0.35; best solution: 1376772
epoch: 638; timestamp: 241.45; epoch's duration:  0.35; best solution: 1376772
epoch: 639; timestamp: 241.79; epoch's duration:  0.34; best solution: 1376772
epoch: 640; timestamp: 242.14; epoch's duration:  0.35; best solution: 1376772
epoch: 641; timestamp: 242.48; epoch's duration:  0.34; best solution: 1376772
epoch: 642; timestamp: 242.82; epoch's duration:  0.34; best solution: 1376772
epoch: 643; timestamp: 243.16; epoch's duration:  0.34; best solution: 1376772
epoch: 644; timestamp: 243.51; epoch's duration:  0.35;

epoch: 736; timestamp: 276.33; epoch's duration:  0.38; best solution: 1376772
epoch: 737; timestamp: 276.69; epoch's duration:  0.36; best solution: 1376772
epoch: 738; timestamp: 277.05; epoch's duration:  0.36; best solution: 1376772
epoch: 739; timestamp: 277.41; epoch's duration:  0.35; best solution: 1376772
epoch: 740; timestamp: 277.77; epoch's duration:  0.36; best solution: 1376772
epoch: 741; timestamp: 278.13; epoch's duration:  0.36; best solution: 1376772
epoch: 742; timestamp: 278.48; epoch's duration:  0.35; best solution: 1376772
epoch: 743; timestamp: 278.84; epoch's duration:  0.36; best solution: 1376772
epoch: 744; timestamp: 279.24; epoch's duration:  0.4; best solution: 1376772
epoch: 745; timestamp: 279.63; epoch's duration:  0.38; best solution: 1376772
epoch: 746; timestamp: 280.0; epoch's duration:  0.37; best solution: 1376772
epoch: 747; timestamp: 280.36; epoch's duration:  0.36; best solution: 1376772
epoch: 748; timestamp: 280.73; epoch's duration:  0.37

In [16]:
print(population[0][0])
for i in range(1, len(population[0])):
    print(population[0][i][0], population[0][i][1], end = " ")
    print(" ".join([str(x) for x in population[0][i][2]]))

4
393 1000 73571 27371 19306 30637 7150 34301 47121 88418 48389 29601 94583 22395 54091 88106 36122 30443 32078 2940 68465 67105 80818 6499 11440 29416 83162 21278 41346 18584 27376 79440 51134 66244 42724 23798 27830 31570 23851 84383 60210 82302 49127 80875 47386 43168 8195 27875 45246 66120 75578 41038 26636 71250 24205 19631 74429 26898 21323 90537 9171 60609 89283 88119 85749 46047 9937 62151 25867 67680 71654 61366 33950 20178 52661 93093 46286 69298 79019 51426 10126 87758 82313 44393 86070 48622 226 91792 38908 33022 31048 36986 83665 96603 98317 14992 79546 63787 3198 76713 48165 88015 15041 55508 45880 36623 88102 78840 90168 83513 65702 48312 9470 46465 52342 2220 89755 34506 79030 74166 50207 99440 33589 78019 3066 2483 22917 84820 62784 50425 78972 70610 66283 17615 38619 55716 59010 63973 12693 34807 2015 7284 41952 40236 37119 87603 83871 62550 12272 96056 79188 63069 63665 20040 79901 89611 14495 13047 31711 55453 63623 45493 24062 5667 1752 18764 22833 95197 19837 2777

In [17]:
print(evaluate_solution(population[0], books_scores))

1376772


In [18]:
print(time() - epochs_duration[0])

299.41370916366577
